
# **🧪 Quality Check (QC)**

Validates **Crash_type (binary or multi-class)**, **Hit and Run**, and **Fatality Risk**.  

## **0) (Optional) Install pandas/numpy (usually preinstalled)**

Installs any missing Python packages so the notebook runs consistently on fresh environments.

In [ ]:
# pip install pandas numpy

## **1) Configure outcome and thresholds**

To know your DB, schema, table name.

In [ ]:
import duckdb
con = duckdb.connect()
con.execute("ATTACH 'gold.duckdb' AS gold (READ_ONLY)")

# === Show all tables (DB, schema, table name) ===
result = con.execute("""
  SELECT database_name, schema_name, table_name
  FROM duckdb_tables()
  WHERE database_name = 'gold'
  ORDER BY 1,2,3
""").df()

print(result)



Using the above details update the TABLE_BY_OUTCOME variable 

Sets the OUTCOME, connects table names to DuckDB, chooses the target column, and defines QC thresholds.

In [ ]:
# --- Imports ---
import os, re, math
from datetime import datetime
import numpy as np
import pandas as pd
import duckdb
from IPython.display import display

# --- Config ---
OUTCOME = "Crash_type"  # or: "Crash_type", "Hit and Run", "Fatality Risk"
DB_PATH = "gold.duckdb"
TABLE_BY_OUTCOME = {
    "Crash_type": 'gold"."main"."gold_crash_type',        # db.schema.Table_name → gold.duckdb.gold.gold_crash_type
    "Hit and Run": 'gold"."gold"."crashes',               # db.schema.Table_name → gold.duckdb.gold.crashes 
    "Fatality Risk": 'gold"."gold"."gold_fatality_risk',  # db.schema.Table_name → gold.duckdb.gold.gold_fatality_risk
}


TABLE_NAME = TABLE_BY_OUTCOME[OUTCOME]

# --- Columns and Targets ---
DEFAULT_TARGET_BY_OUTCOME = {
    "Crash_type": "crash_type",
    "Hit and Run": "hit_and_run_i",
    "Fatality Risk": "fatality_risk"
}
TARGET_COL = None
ID_COL = "crash_record_id"
DATE_COL = "crash_date"

# --- Leakage Hints & Thresholds ---
LEAKAGE_HINTS = {
    "Hit and Run": ["hit_and_run", "hit-run", "hitrun"],
    "Fatality Risk": ["injuries_fatal", "injuries_total", "injuries_incapacitating"],
    "Crash_type": ["crash_type"]
}
# QC thresholds (tune for your dataset size & tolerance)
MISSING_WARN = 0.20   # warn if a column's missing-rate > 20%.
                      # Typical: 0.10 (strict) to 0.30 (lenient). Use ~0.20 for general QC.

HIGH_CARD_LIMIT = 100 # flag categorical columns with >100 unique values as "high-cardinality".
                      # Typical: 50–200. Lower for small datasets; higher if you expect many IDs/buckets.

OUTLIER_RATE_WARN = 0.02  # warn if an individual numeric column has >2% outliers by IQR rule.
                          # Typical: 0.01 (strict) to 0.05 (lenient). Use ~0.02 for balanced sensitivity.


# --- Derived Vars ---
TARGET = (TARGET_COL or DEFAULT_TARGET_BY_OUTCOME.get(OUTCOME))
assert OUTCOME in DEFAULT_TARGET_BY_OUTCOME, f"Unsupported OUTCOME: {OUTCOME}"
print(f"Outcome = {OUTCOME}  |  Target column = {TARGET}")

# --- Load data from DuckDB ---
with duckdb.connect(DB_PATH, read_only=True) as con:
    df = con.execute(f'SELECT * FROM "{TABLE_NAME}"').df()

print(f"Loaded {TABLE_NAME} with shape: {df.shape}")

# --- Date conversion ---
if DATE_COL in df.columns:
    df[DATE_COL] = pd.to_datetime(df[DATE_COL], errors="coerce")

# --- Display sample ---
display(df.head(10))


## **2) Schema & primary key checks**

Confirms required columns exist and verifies crash_record_id is non-null and unique.

In [ ]:
import pandas as pd
issues = issues if 'issues' in globals() else []

REQUIRED_COLUMNS = [
    'crash_record_id', 'weather_condition','lighting_condition'
]

missing_required = [c for c in REQUIRED_COLUMNS if c not in df.columns]
if missing_required:
    issues.append(("SCHEMA", f"Missing required columns: {missing_required}"))
    print("[SCHEMA] Missing:", missing_required)

if 'crash_record_id' not in df.columns:
    issues.append(("KEY", "crash_record_id column missing"))
else:
    null_ids = df['crash_record_id'].isna().sum()
    if null_ids > 0:
        issues.append(("KEY", f"{null_ids} rows have NULL crash_record_id"))
        print("[KEY] NULL crash_record_id rows:", null_ids)
    nunique_ids = df['crash_record_id'].nunique(dropna=True)
    if nunique_ids != len(df):
        approx_dupes = int(len(df) - nunique_ids)
        issues.append(("KEY", f"crash_record_id not unique (duplicates ~ {approx_dupes})"))
        print("[KEY] Non-unique crash_record_id; ~dupes:", approx_dupes)


## **3) QC readiness (no cleaning)**

Validates basic dataframe readiness for QC (consistent names/types, safe coercions) — does not perform cleaning.

In [ ]:

df.columns = [re.sub(r"\s+", "_", c.strip().lower()) for c in df.columns]
ID = ID_COL.lower() if ID_COL else None
DATE = DATE_COL.lower() if DATE_COL else None
TARGET = TARGET.lower() if TARGET else None
print("Target present?", TARGET in df.columns)


## **4) Duplicates**

Checks dataset grain and flags duplicate records or non-unique keys for review.

In [ ]:

issues = []
if ID in df.columns:
    dup_ids = df[ID].value_counts()
    dup_ids = dup_ids[dup_ids > 1]
    if not dup_ids.empty:
        issues.append(("DUPLICATES", f"{dup_ids.shape[0]} duplicated {ID} values"))
        display(dup_ids.head(10).to_frame("count"))
else:
    before = df.shape[0]
    after = df.drop_duplicates().shape[0]
    if before - after > 0:
        issues.append(("DUPLICATES", f"{before - after} fully-duplicate rows"))


## **5) Missingness**

Computes per-column missing rates and highlights columns exceeding the warning threshold.

In [ ]:

miss = (df.isna().mean().rename("missing_rate").reset_index()
          .rename(columns={"index":"column"})
          .sort_values("missing_rate", ascending=False))
display(miss.head(20))
if (miss["missing_rate"] > MISSING_WARN).any():
    issues.append(("MISSINGNESS", "Some columns exceed missingness threshold"))


## **6) Target checks** 

Profiles the target: coerces to binary/ordinal when applicable and stores distribution metadata for reporting.

In [ ]:
import pandas as pd
from typing import Optional, Tuple, Dict

# Canonicalize OUTCOME names so students can use small variants
_OUTCOME_ALIASES = {
    "crash_type": "Crash_type",
    "crash type": "Crash_type",
    "hit&run": "Hit and Run",
    "hit & run": "Hit and Run",
    "fatality_risk": "Fatality Risk",
    "fatality  risk": "Fatality Risk",
}
_outcome_key = OUTCOME.strip()
_outcome_key = _OUTCOME_ALIASES.get(_outcome_key.lower(), _outcome_key)

# Enforce allowed outcomes only
_ALLOWED = {"Crash_type", "Hit and Run", "Fatality Risk"}
if _outcome_key not in _ALLOWED:
    raise ValueError(f"Unsupported OUTCOME: {OUTCOME}. Allowed: {sorted(_ALLOWED)}")
OUTCOME = _outcome_key  # overwrite with canonical value

# --- Helpers (generic) ---
def _normalize_binary_series_generic(s: pd.Series) -> Tuple[pd.Series, Optional[Dict[str,int]]]:
    """
    Coerce common binary encodings to {0,1}. (No KSI terms here.)
    """
    if s is None:
        return s, None
    uniq_raw = set(pd.Series(s.dropna().unique()).tolist())
    if uniq_raw <= {0,1}:
        return s.astype("Int64"), None

    sv = s.astype(str).str.strip().str.lower()
    mapping = {
        "1": 1, "0": 0,
        "true": 1, "false": 0,
        "y": 1, "n": 0,
        "yes": 1, "no": 0,
    }
    mapped = sv.map(mapping)
    if mapped.dropna().isin([0,1]).all() and mapped.notna().any():
        return mapped.astype("Int64"), mapping
    return s, None

def _try_map_fatality_ordinal(s: pd.Series) -> Tuple[Optional[pd.Series], Optional[Dict[str,int]]]:
    """
    Map common severity terms to an ordered 0..K-1 scale for Fatality Risk if present.
    """
    sv = s.astype(str).str.strip().str.lower()
    # If small numeric scale (0..5 etc.), just coerce
    num_try = pd.to_numeric(sv, errors="coerce")
    uniq = sorted(num_try.dropna().unique())
    if 2 < len(uniq) <= 8 and (pd.Series(uniq).max() <= 10):
        return num_try.astype("Int64"), {"numeric_scale_detected": True}
    # Text buckets (customize as needed)
    lut = {
        "none": 0, "no injury": 0, "no fatality": 0,
        "low": 1, "minor": 1,
        "medium": 2, "moderate": 2, "med": 2,
        "high": 3, "serious": 3, "severe": 3,
        "fatal": 4, "death": 4
    }
    mapped = sv.map(lut)
    if mapped.notna().any():
        return mapped.astype("Int64"), lut
    return None, None

# --- Target analysis starts here ---
target_info = {}
if TARGET and TARGET in df.columns:
    col = TARGET
    nonnull = df[col].dropna()
    uniq_vals = sorted(pd.Series(nonnull.unique()).tolist(), key=lambda x: str(x))
    k = len(uniq_vals)

    target_info = {
        "name": col,
        "k": k,
        "values": uniq_vals[:12]
    }

    expects_binary = False
    if OUTCOME == "Hit and Run":
        expects_binary = True
    elif OUTCOME == "Crash_type":
        # Crash_type can be binary (2 string labels) or multiclass
        expects_binary = (k == 2)
        if k > 30:
            issues.append(("TARGET", f"{k} categories — consider grouping rare levels for Crash_type"))
    elif OUTCOME == "Fatality Risk":
        # Typically binary (KSI vs Non-KSI) or ordinal severity; try binary if k==2, else try ordinal
        expects_binary = (k == 2)

    series_for_balance = None

    # ----- Outcome-specific handling -----
    if OUTCOME == "Fatality Risk" and k > 2:
        # Try ordinal mapping first
        ord_series, ord_map = _try_map_fatality_ordinal(df[col])
        if ord_series is not None:
            ord_col = col + "_ord"
            df[ord_col] = ord_series
            target_info["ordinal_column"] = ord_col
            target_info["interpreted_as"] = "ordinal"
            target_info["ordinal_mapping"] = ord_map
        else:
            # Try binary mapping using common KSI strings (scoped here only)
            sv = df[col].astype(str).str.strip().str.lower()
            ksi_map = {
                "ksi": 1, "k.s.i": 1,
                "non-ksi": 0, "non ksi": 0, "non_ksi": 0, "non ksi ": 0
            }
            mapped = sv.map(ksi_map)
            if mapped.dropna().isin([0,1]).all() and mapped.notna().any():
                bin_col = col + "_bin"
                df[bin_col] = mapped.astype("Int64")
                target_info["binary_column"] = bin_col
                target_info["interpreted_as"] = "binary"
                issues.append(("TARGET", f"Normalized `{col}` to {{0,1}} using KSI mapping → `{bin_col}`"))
                series_for_balance = df[bin_col]
            else:
                issues.append(("TARGET", f"`{col}` has {k} distinct values — expected binary/ordinal for Fatality Risk"))

    # Binary normalization for outcomes that should be binary
    if expects_binary and series_for_balance is None:
        s_norm, used_map = _normalize_binary_series_generic(df[col])
        uniq_after = set(pd.Series(s_norm.dropna().unique()).tolist())
        if uniq_after <= {0,1}:
            bin_col = col + "_bin"
            df[bin_col] = s_norm
            target_info["binary_column"] = bin_col
            target_info["interpreted_as"] = "binary"
            if used_map:
                issues.append(("TARGET", f"Normalized `{col}` to {{0,1}} using mapping {used_map} → `{bin_col}`"))
            series_for_balance = df[bin_col]
        else:
            if k == 2:
                issues.append(("TARGET", f"`{col}` is binary but uses non 0/1 encodings: {uniq_vals}"))
            else:
                # For Crash_type multiclass, we accept; for Fatality Risk we warned above.
                pass

    # Always store distribution (top 20)
    dist = df[col].value_counts(dropna=False).to_frame("count")
    target_info["distribution"] = dist.head(20)

    # Balance metrics for binary targets (from series_for_balance if set)
    if series_for_balance is not None:
        counts = series_for_balance.value_counts(dropna=False)
        total = int(counts.sum())
        pos = int(counts.get(1, 0))
        neg = int(counts.get(0, 0))
        minority = min(pos, neg)
        majority = max(pos, neg)
        minority_rate = (minority / total) if total else 0.0
        imbalance_ratio = (majority / minority) if minority > 0 else float("inf")
        target_info["class_balance"] = {
            "total": total,
            "pos": pos,
            "neg": neg,
            "minority_rate": minority_rate,
            "imbalance_ratio": imbalance_ratio
        }
        # keep your chosen threshold; here 10%
        if minority_rate < 0.10:
            issues.append(("IMBALANCE", f"Minority class <10% (minority_rate={minority_rate:.3f}). Consider resampling/thresholding."))

    # Optional: if ordinal Fatality Risk was mapped, show level frequencies too
    if OUTCOME == "Fatality Risk" and target_info.get("ordinal_column"):
        ord_dist = df[target_info["ordinal_column"]].value_counts(dropna=False).sort_index().to_frame("count")
        target_info["ordinal_distribution"] = ord_dist

    # Display summary for the notebook run
    print(f"### Target `{col}` (k={k})")
    display(target_info["distribution"])
    cb = target_info.get("class_balance")
    if isinstance(cb, dict):
        print("\nClass balance (binary):")
        print(f"- total={cb['total']}, pos={cb['pos']}, neg={cb['neg']}")
        print(f"- minority_rate={cb['minority_rate']:.3f}, imbalance_ratio={cb['imbalance_ratio']:.2f}")
    if OUTCOME == "Fatality Risk" and target_info.get("ordinal_distribution") is not None:
        print("\nOrdinal distribution (mapped):")
        display(target_info["ordinal_distribution"])

else:
    issues.append(("TARGET", f"Target `{TARGET}` not found"))
    print(f"[WARN] Target `{TARGET}` not found.")


## **7) Class Imbalance Indicator**
Reports class counts/percentages and flags severe imbalance when the minority rate is too low.

In [ ]:
import pandas as pd
issues = issues if 'issues' in globals() else []

if TARGET and TARGET in df.columns:
    vc = df[TARGET].value_counts(dropna=False)
    vc_df = vc.rename_axis("class").to_frame("count")
    vc_df["pct"] = (vc_df["count"] / len(df)).round(4)

    # Show distribution once
    try:
        display(vc_df)
    except Exception:
        print(vc_df)

    # Persist into target_info for final report
    target_info = locals().get("target_info", {})
    target_info["name"] = TARGET
    target_info["k"] = int(df[TARGET].nunique(dropna=True))
    target_info["distribution"] = vc_df.reset_index()

    # Determine how many classes (ignoring NaN)
    levels = [cls for cls in vc.index.tolist() if pd.notna(cls)]
    n_classes = len(levels)

    # Optional: allow a user-configured positive class label
    POSITIVE_LABEL = locals().get("POSITIVE_LABEL", None)

    if n_classes == 2:
        # Treat as binary even if labels are strings
        if POSITIVE_LABEL in levels:
            pos_name = POSITIVE_LABEL
            neg_name = levels[0] if levels[1] == POSITIVE_LABEL else levels[1]
        else:
            # pick minority as positive by default
            pos_name = vc.idxmin()
            neg_name = vc.idxmax()

        pos = int(vc.get(pos_name, 0))
        neg = int(vc.get(neg_name, 0))
        total = int(len(df))
        pos_rate = (pos / total) if total else 0.0

        # flag imbalance if extreme (<20% or >80% positive)
        if pos_rate < 0.20 or pos_rate > 0.80:
            issues.append(("IMBALANCE", f"Target '{TARGET}' imbalanced (positive='{pos_name}', rate={pos_rate:.3f})"))
            print(f"[IMBALANCE] Binary target detected: positive='{pos_name}', rate={pos_rate:.3f}")
        else:
            print(f"[IMBALANCE] Binary target detected: positive='{pos_name}', rate={pos_rate:.3f} (looks OK)")

        # store detailed balance for final report
        minority = min(pos, neg)
        target_info["class_balance"] = {
            "total": total,
            "pos": pos,
            "neg": neg,
            "pos_label": str(pos_name),
            "neg_label": str(neg_name),
            "minority_rate": (minority / total) if total else 0.0,
            "imbalance_ratio": (max(pos, neg) / max(1, minority)) if minority else float("inf"),
        }

    elif n_classes > 2:
        # Multi-class: just show top classes (already displayed above)
        print("[IMBALANCE] Multi-class target — review class distribution above.")
    else:
        print("[IMBALANCE] Target has <2 non-null classes; nothing to assess.")
    
    # write back
    locals()["target_info"] = target_info

else:
    print("[IMBALANCE] Skipped: TARGET not set or missing from df.")


## **8) Boolean Normalization Audit for _i Flags**

Audits all *_i indicators and warns if any use encodings outside {0,1,NaN}.

In [ ]:
import pandas as pd

issues = issues if 'issues' in globals() else []

# Consider any column ending with "_i" as an indicator/boolean.
flag_cols = [c for c in df.columns if c.endswith('_i')]

bad_flags = {}
for c in flag_cols:
    # look only at observed, non-null values
    uniq = set(pd.Series(df[c].dropna().unique()).tolist())
    # allow only {0,1}; NaN is allowed separately
    if not (uniq <= {0, 1}):
        bad_flags[c] = sorted(list(uniq))[:12]  # sample first few values

if bad_flags:
    issues.append(("BOOLEANS", f"Non-binary encodings in flags: {list(bad_flags.keys())}"))
    print("[BOOLEANS] Non {0,1} values detected in:")
    for k, v in bad_flags.items():
        print(f"  - {k}: examples -> {v}")
else:
    print("[BOOLEANS] All *_i flags are in {{0,1,NaN}}")


## **9) Leakage** 

Surfaces name-based suspects and enforces removal of forbidden leakage/ID columns before modeling.

In [ ]:
import re
issues = issues if 'issues' in globals() else []

# 0) Outcome-specific name hints 
if TARGET and TARGET in df.columns:
    hints = [h.lower() for h in LEAKAGE_HINTS.get(OUTCOME, [])]
    found_hint = [c for c in df.columns if any(h in c.lower() for h in hints) and c != TARGET]
    if found_hint:
        issues.append(("LEAKAGE", f"Suspected leakage columns (name hints): {found_hint[:12]}{'...' if len(found_hint)>12 else ''}"))
        display(pd.DataFrame({'suspected_leakage_cols': found_hint}).assign(source="hints"))

# 1) Expanded common leakage/ID/post-event list (flag only)
COMMON_LEAKAGE_EXACT = {
    # already had some of these
    "report_type", "photos_taken_i", "statements_taken_i", "date_police_notified",
    "veh_vehicle_id_list_json", "ppl_person_id_list_json", "location_json", "street_name",
    # more admin/ID/meta
    "rd_no", "report_number", "case_no", "case_number", "case_status", "case_disposition",
    "investigation_status", "report_completed_i",
    "date_reported", "time_reported", "time_police_notified",
    # adjudication / enforcement
    "citation", "citation_number", "violation_code", "ticket_number", "arrest_flag", "charge_code",
    # medical / EMS / post-crash response
    "ems_run_number", "ambulance_id", "hospital_name", "hospital_code", "trauma_center",
    # file/meta columns that shouldn’t be model features
    "created_at", "updated_at", "loaded_at", "ingest_time", "etl_batch_id", "source_file", "row_hash"
}

# 2) Regex patterns to catch families of leakage columns (flag only)
COMMON_LEAKAGE_PATTERNS = [
    # IDs & JSON aggregates
    r"_id_list_json$", r"\b[a-z]+_id\b", r"\bveh_.*_id\b", r"\bppl_.*_id\b",
    # enforcement & adjudication
    r"\bcitation", r"\bticket", r"\bviolation", r"\barrest", r"\bcharge",
    # post-event admin times/dates
    r"date_.*(reported|notified)", r"time_.*(reported|notified)",
    # injuries / severity (likely label leakage for certain outcomes)
    r"\binjur", r"incapacitating|serious|severe|fatal|death|deceased|pronounced",
    # tow-related (post-crash response)
    r"\btow(ed|ing)?\b", r"vehicle_?_?towed", r"num_vehicles_towed", r"tow_company|tow_destination",
    # medical/EMS
    r"\bems\b|\bambulance\b|\bhosp|\btrauma|\ber\b|\bmedic",
    # meta/ETL
    r"^etl_", r"^ingest_", r"^load_", r"_ingested$", r"_loaded$"
]

# 3) Scan columns
cols = df.columns.tolist()
suspects_exact = [c for c in cols if c in COMMON_LEAKAGE_EXACT]
rx = re.compile("|".join(COMMON_LEAKAGE_PATTERNS), flags=re.IGNORECASE)
suspects_regex = [c for c in cols if rx.search(c)]

# De-dup + remove true target
suspects = sorted(set(suspects_exact) | set(suspects_regex))
if TARGET in suspects:
    suspects.remove(TARGET)

if suspects:
    print("[LEAKAGE] Common leakage/ID/post-event suspects:")
    display(pd.DataFrame({"suspected_leakage_cols": suspects}).assign(source="common"))
    issues.append(("LEAKAGE", f"Common leakage/ID/post-event suspects: {suspects[:20]}{'...' if len(suspects)>20 else ''}"))
else:
    print("[LEAKAGE] No common leakage/ID/post-event columns matched.")

# 4) Keeping “FORBIDDEN” list as a strict subset 
FORBIDDEN = {
    "report_type","photos_taken_i","statements_taken_i","date_police_notified",
    "veh_vehicle_id_list_json","ppl_person_id_list_json","location_json","street_name"
}
present_forbidden = [c for c in FORBIDDEN if c in df.columns and c != TARGET]
if present_forbidden:
    issues.append(("LEAKAGE", f"Forbidden (leakage/IDs) present: {present_forbidden}"))
    print("[LEAKAGE] Forbidden present:", present_forbidden)
else:
    print("[LEAKAGE] No forbidden columns detected (strict list).")


## **10) Derived Time Features Present**

Verifies year, month, day_of_week, hour, is_weekend are available; shows date range and latest crash date.


In [ ]:
issues = issues if 'issues' in globals() else []
expected_time_cols = ['year','month','day_of_week','hour','is_weekend']
missing_time = [c for c in expected_time_cols if c not in df.columns]
if missing_time:
    issues.append(("TIME", f"Missing derived time columns: {missing_time}"))
    print("[TIME] Missing:", missing_time)
else:
    print("[TIME] All derived time columns present")


## **11) Geo Bins & grid_id Consistency**

Checks that grid_id numerically matches lat_bin/lng_bin (ignoring formatting) and reports any mismatches.

In [ ]:
import pandas as pd
issues = issues if 'issues' in globals() else []
expected_geo = ['lat_bin','lng_bin','grid_id']
missing_geo = [c for c in expected_geo if c not in df.columns]
if missing_geo:
    issues.append(("GEO", f"Missing geo bin columns: {missing_geo}"))
    print("[GEO] Missing:", missing_geo)
else:
    derived = df['lat_bin'].astype(str).str.strip() + "_" + df['lng_bin'].astype(str).str.strip()
    mismatch = (df['grid_id'].astype(str).str.strip() != derived).sum()
    if mismatch > 0:
            print("[GEO] grid_id matches lat/lng bins")


## **12) Categorical cardinality & Outliers**

Flags high-cardinality categoricals to compress and runs an IQR-based scan for numeric outliers.

In [ ]:

# Categorical
cat_cols = [c for c in df.columns if df[c].dtype == 'object']
cat_card = pd.DataFrame({'column': cat_cols, 'unique_values': [df[c].nunique(dropna=True) for c in cat_cols]}).sort_values('unique_values', ascending=False)
display(cat_card.head(20))
if (cat_card['unique_values'] > HIGH_CARD_LIMIT).any():
    issues.append(("CARDINALITY", "High-cardinality categoricals present"))

# Outliers (IQR)
num_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
out_rows = []
for c in num_cols:
    s = df[c].dropna()
    if s.empty: 
        continue
    q1, q3 = s.quantile(0.25), s.quantile(0.75)
    iqr = q3 - q1
    if not pd.isna(iqr) and iqr != 0:
        lo, hi = q1 - 1.5*iqr, q3 + 1.5*iqr
        rate = ((df[c] < lo) | (df[c] > hi)).mean()
        rate = float(rate) if not pd.isna(rate) else 0.0
        out_rows.append((c, rate))
out_df = pd.DataFrame(out_rows, columns=['column','outlier_rate']).sort_values('outlier_rate', ascending=False)
display(out_df.head(20))
if (out_df['outlier_rate'] > OUTLIER_RATE_WARN).any():
    issues.append(("OUTLIERS", "Some numeric columns exceed outlier threshold"))
issues = issues if 'issues' in globals() else []
violations = []

def _max_over(col, k):
    if col in df.columns and pd.api.types.is_numeric_dtype(df[col]):
        mx = pd.to_numeric(df[col], errors='coerce').max()
        if pd.notna(mx) and mx > k:
            violations.append((col, float(mx), k))

CAPS = {'veh_count': 5, 'ppl_count': 10, 'injuries_total': 20}
for col, cap in CAPS.items():
    _max_over(col, cap)

age_cols = [c for c in df.columns if 'age' in c and pd.api.types.is_numeric_dtype(df[c])]
for c in age_cols:
    mx = pd.to_numeric(df[c], errors='coerce').max()
    if pd.notna(mx) and mx > 110:
        violations.append((c, float(mx), 110))

if violations:
    issues.append(("OUTLIERS_DOMAIN", f"Domain caps exceeded: {violations[:10]}"))
    print("[OUTLIERS_DOMAIN] Exceeded caps:", violations[:10])
else:
    print("[OUTLIERS_DOMAIN] All domain caps satisfied")


## **13) Gold Non-Redundancy (Table-Level Duplicate Check)**

Queries DuckDB to confirm no duplicate crash_record_id are present in the Gold table.

In [ ]:
issues = issues if 'issues' in globals() else []
try:
    DB_PATH  # noqa
    TABLE_NAME  # noqa
except NameError:
    print("[GOLD] Skipped: DB_PATH or TABLE_NAME is not defined in this notebook.")
else:
    try:
        import duckdb
        with duckdb.connect(DB_PATH, read_only=True) as con:
            total_rows = con.execute(f'SELECT COUNT(*) FROM "{TABLE_NAME}"').fetchone()[0]
            distinct_ids = con.execute(f'SELECT COUNT(DISTINCT crash_record_id) FROM "{TABLE_NAME}"').fetchone()[0]
        if total_rows != distinct_ids:
            issues.append(("GOLD_DUPES", f'Gold table "{TABLE_NAME}" has duplicate crash_record_id rows '
                                         f'(total={total_rows}, distinct={distinct_ids})'))
            print(f'[GOLD] WARNING: total_rows={total_rows}, distinct_ids={distinct_ids}')
        else:
            print(f'[GOLD] OK: Unique crash_record_id enforced in {TABLE_NAME}')
    except Exception as e:
        issues.append(("GOLD_CHECK_ERROR", f"Error while checking Gold table: {e!r}"))
        print("[GOLD] Error:", repr(e))


## **14) Final findings**

Generates a concise narrative QC report summarizing findings and prioritized next actions for modeling readiness.

In [ ]:
# === Final Narrative QC Report  ===
from datetime import datetime
import pandas as pd

def _pct(x):
    try:
        return f"{100*x:.1f}%"
    except Exception:
        return "-"

# Safely grab prior-cell variables (with defaults)
issues      = locals().get("issues", [])
df          = locals().get("df")
ID          = locals().get("ID", None)
DATE        = locals().get("DATE", None)
OUTCOME     = locals().get("OUTCOME", "<unset>")
TARGET      = locals().get("TARGET", "<unset>")
miss        = locals().get("miss", pd.DataFrame(columns=["column","missing_rate"]))
hi_card     = locals().get("hi_card", pd.DataFrame(columns=["column","unique_values"]))
flag_out    = locals().get("flag_out", pd.DataFrame(columns=["column","outlier_rate"]))
corr_pairs  = locals().get("corr_pairs", pd.DataFrame(columns=["col_a","col_b","pearson_r"]))
leak_cols   = locals().get("leak_cols", [])
date_range  = locals().get("date_range", None)      # (min_ts, max_ts)
geo_stats   = locals().get("geo_stats", None)       # {"lat_ok": float, "lon_ok": float}
target_info = locals().get("target_info", {})       # {"name","k","values","interpretation","distribution","class_balance"}
dup_preview = locals().get("dup_preview", None)     # sample duplicate IDs

print(f"# QC Summary — {datetime.now().strftime('%Y-%m-%d %H:%M')}\n")

# ---------------------------
# Dataset
# ---------------------------
print("## Dataset")
if df is None:
    raise RuntimeError("`df` was not found. Run earlier cells first.")
print(f"Rows × Cols: {df.shape[0]:,} × {df.shape[1]}")
print(f"ID column: {ID if (isinstance(ID, str) and ID in df.columns) else '—'}")
print(f"Date column: {DATE if (isinstance(DATE, str) and DATE in df.columns) else '—'}")
print(f"Outcome: {OUTCOME}  |  Target: {TARGET}")
latest_crash_date = None
if isinstance(DATE, str) and (df is not None) and (DATE in df.columns):
    _dt = pd.to_datetime(df[DATE], errors="coerce")
    if _dt.notna().any():
        latest_crash_date = _dt.max()
print(f"Latest crash date:", latest_crash_date)

# ---------------------------
# Target Analysis
# ---------------------------
print("## Target Analysis")
if target_info.get("name"):
    print(f"Target column `{target_info['name']}` with {target_info.get('k','?')} distinct non-null values")
    if target_info.get("interpretation"):
        print(f"Interpreted as: {target_info['interpretation']}")
    if target_info.get("values"):
        print("Example values:", target_info["values"])
    if isinstance(target_info.get("distribution"), pd.DataFrame):
        print("\nTop of target distribution:")
        print(target_info["distribution"].to_string())
    cb = target_info.get("class_balance")
    if isinstance(cb, dict):
        print("\nClass balance (binary):")
        print(f"- total={cb.get('total','?')}, pos={cb.get('pos','?')}, neg={cb.get('neg','?')}")
        if "minority_rate" in cb:
            print(f"- minority_rate={cb['minority_rate']:.3f}, imbalance_ratio={cb.get('imbalance_ratio','?')}")
else:
    print("Target not found or not analyzed in prior cells.")
print()

# ---------------------------
# Schema & Primary Key
# ---------------------------
print("## Schema & Primary Key")
schema_msgs = [m for (t,m) in issues if t in {"SCHEMA","KEY"}]
if schema_msgs:
    for m in schema_msgs:
        print("-", m)
else:
    print("Required columns present and primary key looks valid.")
print()

# ---------------------------
# Boolean Flags (*_i)
# ---------------------------
print("## Boolean Flags (*_i)")
bool_msgs = [m for (t,m) in issues if t == "BOOLEANS"]
if bool_msgs:
    for m in bool_msgs:
        print("-", m)
else:
    print("All *_i flags appear to be normalized to {0,1,NaN} or no *_i flags found.")
print()

# ---------------------------
# Duplicates
# ---------------------------
print("## Duplicates")
dup_msgs = [m for (t,m) in issues if t == "DUPLICATES"]
if dup_msgs:
    for m in dup_msgs:
        print("-", m)
    if dup_preview is not None:
        print("\nSample duplicate IDs:")
        print(dup_preview.to_string())
else:
    print("No duplicate issues detected.")
print()

# ---------------------------
# Missingness
# ---------------------------
print("## Missingness")
if not miss.empty:
    top_miss = miss.head(10).copy()
    top_miss["missing_rate"] = (top_miss["missing_rate"]*100).round(1).astype(str) + "%"
    print("Top 10 columns by missing rate:")
    print(top_miss.to_string(index=False))
else:
    print("No missingness table available.")
print()

# ---------------------------
# Categorical Cardinality
# ---------------------------
print("## Categorical Cardinality")
if not hi_card.empty:
    print(f"{hi_card.shape[0]} categorical columns exceed the high-cardinality threshold.")
    print("Top offenders:")
    print(hi_card.head(10).to_string(index=False))
else:
    print("No high-cardinality categorical columns flagged.")
print()

# ---------------------------
# Numeric Outliers (IQR)
# ---------------------------
print("## Numeric Outliers (IQR)")
if not flag_out.empty:
    show = flag_out.head(10).copy()
    show["outlier_rate"] = (show["outlier_rate"]*100).round(1).astype(str) + "%"
    print(f"{flag_out.shape[0]} numeric columns exceed the outlier threshold.")
    print(show.to_string(index=False))
else:
    print("No numeric columns exceeded the outlier threshold.")
print()

# ---------------------------
# Domain Caps (veh/ppl/injuries/age)
# ---------------------------
print("## Domain Caps")
dom_msgs = [m for (t,m) in issues if t == "OUTLIERS_DOMAIN"]
if dom_msgs:
    for m in dom_msgs:
        print("-", m)
else:
    print("All domain cap checks passed (veh_count, ppl_count, injuries_total, age bounds).")
print()

# ---------------------------
# Date & Geo
# ---------------------------
print("## Date & Geo")
if isinstance(date_range, tuple) and len(date_range) == 2:
    print("Date range:", date_range[0], "→", date_range[1])
else:
    print("No date column parsed or not available.")
if isinstance(geo_stats, dict) and {"lat_ok","lon_ok"} <= geo_stats.keys():
    print(f"Geo bounds OK fraction — lat: {_pct(geo_stats['lat_ok'])}, lon: {_pct(geo_stats['lon_ok'])}")
else:
    print("No lat/lon bounds check or columns not present.")
time_msgs = [m for (t,m) in issues if t == "TIME"]
if time_msgs:
    for m in time_msgs:
        print("-", m)
print()

# ---------------------------
# Leakage
# ---------------------------
print("## Leakage")
if leak_cols:
    print("Suspected (name-based) leakage columns:")
    print(pd.DataFrame({"suspected_leakage_cols": leak_cols}).head(20).to_string(index=False))
else:
    print("No suspected leakage columns by name heuristic.")
leak_enforce = [m for (t,m) in issues if t == "LEAKAGE"]
if leak_enforce:
    for m in leak_enforce:
        print("-", m)
print()

# ---------------------------
# Strong Numeric Correlations
# ---------------------------
# This section only prints if corr_pairs exists and is non-empty.
print("## Strong Numeric Correlations (|r| ≥ 0.7)")
if isinstance(corr_pairs, pd.DataFrame) and not corr_pairs.empty:
    print(corr_pairs.head(20).to_string(index=False))
else:
    print("None detected above threshold or skipped in this QC.")
print()

# ---------------------------
# Gold Layer (Non-Redundancy)
# ---------------------------
print("## Gold Layer")
gold_msgs = [m for (t,m) in issues if t in {"GOLD_DUPES","GOLD_CHECK_ERROR"}]
if gold_msgs:
    for m in gold_msgs:
        print("-", m)
else:
    print("Gold table check: no duplicate crash_record_id detected (or check skipped).")
print()

# ---------------------------
# Recommended Next Actions
# ---------------------------
print("## Recommended Next Actions")
recommendations = []
for tag, msg in issues:
    t = tag.upper()
    m = msg.lower()
    if t == "TARGET" and ("binary" in m or "0/1" in m or "encode" in m):
        recommendations.append("Map the target to {0,1} and document the mapping used.")
    if t == "MISSINGNESS":
        recommendations.append("Impute or drop columns with high missingness; document choices and thresholds.")
    if t == "CARDINALITY":
        recommendations.append("Reduce high-cardinality categoricals (group rare levels, hashing, or target encoding).")
    if t == "OUTLIERS":
        recommendations.append("Winsorize/clip, transform, or validate data entry for flagged numeric columns.")
    if t == "OUTLIERS_DOMAIN":
        recommendations.append("Enforce domain caps (veh/ppl/injuries/age) or justify exceptions with domain notes.")
    if t == "DUPLICATES":
        recommendations.append("Resolve duplicates with a consistent rule (latest-wins/aggregate) and de-dup the dataset.")
    if t == "LEAKAGE":
        recommendations.append("Drop or justify suspected leakage columns; ensure only report-time features remain.")
    if t == "BOOLEANS":
        recommendations.append("Normalize *_i flags to {0,1} and re-run QC to confirm.")
    if t == "TIME":
        recommendations.append("Add derived time fields (year, month, day_of_week, hour, is_weekend).")
    if t == "GEO":
        recommendations.append("Ensure lat_bin/lng_bin exist and grid_id = f(lat_bin,lng_bin).")
    if t == "IMBALANCE":
        recommendations.append("Handle class imbalance (class weights, resampling, threshold tuning, or focal loss).")
    if t in {"SCHEMA","KEY"}:
        recommendations.append("Satisfy required schema and enforce a unique, non-null primary key.")

if not recommendations:
    print("- Dataset passes current thresholds; proceed to feature engineering.")
else:
    seen = set()
    for r in recommendations:
        if r not in seen:
            seen.add(r)
            print("-", r)

print("\n— End of QC Report —")


## **📘 Assignment: Convert Your QC Summary into a Professional Data Quality Report**

**Goal.** Using the QC summary you just generated, write a **Professional Data Quality Report** for your chosen outcome. 


### **📦 Deliverables**
- **Report (PDF)**


### **🧭 Required Structure (1-2 pages main report)**
1) **Executive Summary**  
   - One-paragraph verdict on overall data readiness for ML.  

2) **Dataset & Target Overview**  
   - Rows × columns, ID column, Date column, **Outcome** and **Target** (state the exact column name).  
   - Briefly characterize the target (binary/multiclass; class balance: minority %, imbalance ratio).

3) **Quality Assessment (evidence-based)**  
   Summarize the QC sections **as claims + evidence** (You can cite metrics from your QC output):  
   - **Schema & Primary Key:** required fields present? PK unique & non-null?  
   - **Missingness:** top missing columns (with %), plan to impute/drop.  
   - **Categorical Cardinality:** any high-card columns and your plan to reduce them.  
   - **Numeric Outliers (IQR) & Domain Caps:** which columns exceeded caps; justify clips vs. investigation.  
   - **Time & Geo:** date range, latest crash date, derived time fields present, consistent with bins.  
   - **Leakage:** confirm forbidden columns removed; mention any name-based suspects and actions.  
   - **Gold Non-Redundancy:** duplicate key status for the Gold table.

4) **Risks & Ethics (2–4 bullets)**  
   - Potential biases (e.g., missingness patterns by location/time) and impact on model fairness.  
   - Consequences of leakage or mis-specified grain.

5) **Recommendations & Next Actions**  
   - State **go/no-go for modeling** and what must be done before training.


### **🧰 Submission Notes**
- Keep the main report **1-2 pages**.  
- Use consistent notation, and labeled figures/tables.  
- If you re-ran QC after fixes, **state what changed**.

> **Reminder:** Your report must be **self-contained**. A reviewer should understand the dataset’s fitness for modeling without opening the notebook.